# extract text and its position

In [1]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter, HTMLConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO


def convert_pdf_to_txt(path1, path2, num = 0):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'                                                                                                                                                 
    laparams = LAParams()
    device = HTMLConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path1, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for i, page in enumerate(PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True)):
        if num > 0 and i > num:
            break
        interpreter.process_page(page)

    text = retstr.getvalue()
    file_name = 'page' + str(num)
    f = open(path2+file_name, 'w')
    f.write(text)
    f.close()
    #text = retstr.getvalue()
    fp.close()
    device.close()
    retstr.close()




# set paths

In [2]:
#from Tkinter import *
#import tkFileDialog
from tkinter import *
import tkinter.filedialog

win = Tk()
win.title('Select the PDF file')
var = StringVar()

w = Label(win, text="File Path:")
e = Entry(win, textvariable=var)
#b = Button(win, text="Browse",command=lambda:var.set(tkFileDialog.askopenfilename()))
b = Button(win, text="Browse",command=lambda:var.set(tkinter.filedialog.askopenfilename()))
w.pack(side=LEFT)
e.pack(side=LEFT)
b.pack(side=LEFT)
win.mainloop()

path1 = var.get()

win = Tk()
win.title('Select the output file\'s direnctory')
var = StringVar()

w = Label(win, text="File Path:")
e = Entry(win, textvariable=var)
#b = Button(win, text="Browse",command=lambda:var.set(tkFileDialog.askdirectory()))
b = Button(win, text="Browse",command=lambda:var.set(tkinter.filedialog.askdirectory()))
w.pack(side=LEFT)
e.pack(side=LEFT)
b.pack(side=LEFT)
win.mainloop()

path2 = var.get()+'/'

In [3]:
#path2 = 'C:/Users/zhongzhu/Documents/GitHub/work/temp/'
#path1 = 'C:/Users/zhongzhu/Documents/GitHub/work/Seattle.pdf'
#print(path1, path2)

In [4]:
convert_pdf_to_txt(path1, path2)

# record positions

In [3]:
import re

def record_pos(path, page=0, LINE_HEIGHT = 1, start_page=0, end_page=None):
    f = open(path+'page'+str(page), 'r')
    pages = f.read().split('#PAGE')
    f.close()
    
    contents = {}
    for page in pages[start_page: end_page]:
        page_num = page[page.find('<<<')+3:page.find('>>>')]
        page_, num = re.subn(r'<<<.*?>>>', '',page)
    
        boxes = page_.split('TEXT_BOX')

        for content in boxes:
            c = content.split(';')
            if(c[0].find('left:') > 0): #if not empty
                l = c[0][c[0].find('left:')+5:c[0].find('px')]
                t = c[1][c[1].find('top:')+4:c[1].find('px')]
                w = c[2][c[2].find('width:')+6:c[2].find('px')]
                h = c[3][c[3].find('height:')+7:c[3].find('px')]

                text = "".join(c[4:])
                #contents[text.decode('utf-8')] = [(int(l), int(t), int(w), int(h))] #whole box
                lines = text.split('\n')

                tempCount = 0
                for line in lines:
                    if re.search(r'[A-Za-z0-9,.?!]', line):
                        if not line in contents:
                            contents[line] = [(int(l), int(t)+tempCount, int(w), int(h), page_num)]
                        else:
                            contents[line].append((int(l), int(t)+tempCount, int(w), int(h), page_num))
                    tempCount += LINE_HEIGHT ### default value 1
    return contents


In [4]:
contents = record_pos(path2)
#for pos in contents:
    #print(pos)
    #print(contents[pos])

In [5]:
#import Tkinter
#import tkSimpleDialog
import tkinter
import tkinter.simpledialog

#root = Tkinter.Tk()
root = tkinter.Tk()
root.title('select sample pages')
var1 = tkinter.StringVar()
var2 = tkinter.StringVar()
#b1 = Button(root, text="Select start page",command=lambda:var1.set(tkSimpleDialog.askinteger("Select start page", "enter page number", parent = root)))
#b2 = Button(root, text="Select end page",command=lambda:var2.set(tkSimpleDialog.askinteger("Select end page", "enter page number", parent = root)))
b1 = tkinter.Button(root, text="Select start page",command=lambda:var1.set(tkinter.simpledialog.askinteger("Select start page", "enter page number", parent = root)))
b2 = tkinter.Button(root, text="Select end page",command=lambda:var2.set(tkinter.simpledialog.askinteger("Select end page", "enter page number", parent = root)))
b1.grid(column = 0, row = 0)
b2.grid(column = 1, row = 0)
#var = tkSimpleDialog.askstring("Select Pates", "enter your name", parent = root)
root.mainloop()
start_page = var1.get()
end_page = var2.get()

In [6]:
# get the positions of some sample pages
first_page_contents = record_pos(path2, start_page=int(start_page), end_page=int(end_page))

In [7]:
#for i in first_page_contents:
    #print(i)
    #print(first_page_contents[i])

In [8]:
filted_contents = {}
for text in contents:
    if len(contents[text]) > 60:  ###set at beginning#####################
        filted_contents[text] = contents[text]
        #print(text + ' ' +str(len(filted_contents[text])))

In [9]:
#from Tkinter import *
#import Tkinter as tk
import tkinter as tk

class Example(tk.Frame):
     
    def __init__(self, root, *args, **kwargs):
        tk.Frame.__init__(self, root, *args, **kwargs)
        self.root = root
        self.vsb = tk.Scrollbar(self, orient="vertical")
        self.text = tk.Text(self, width=40, height=20, yscrollcommand=self.vsb.set)
        self.vsb.config(command=self.text.yview)
        self.vsb.pack(side="right", fill="y")
        self.text.pack(side="left", fill="both", expand=True)
        
        self.temp_contents = {}

        var = []
        c = []
        for i, cat in enumerate(sorted(filted_contents)):
            c.append(cat)
            v = tk.BooleanVar()
            var.append(v)
            cb = tk.Checkbutton(self, text="#%s" % cat, variable=var[i])
            self.text.window_create("end", window=cb)
            self.text.insert("end", "\n") # to force one checkbox per line
                      
        def var_states():
            for i,v in enumerate(var):
                if v.get():
                    #print(c[i])
                    self.temp_contents[c[i]] = filted_contents[c[i]]
        tk.Button(root, text='Select', command=var_states).pack()
        
root = tk.Tk()
root.title('selecte categories')
win = Example(root)
win.pack(side="top", fill="both", expand=True)
root.mainloop()

filted_contents = win.temp_contents

# search neighbors

In [10]:
from itertools import product
neighbors = {}
    
for line1, line2 in product(filted_contents, first_page_contents):
    key = line1
    value = line2
    if not key in neighbors:
        neighbors[key] = []
    if(line1 != line2 and line1 in first_page_contents
       and abs(int(first_page_contents[line1][0][0])-int(first_page_contents[line2][0][0]))<1000    ###set at beginning
       and abs(int(first_page_contents[line1][0][1])-int(first_page_contents[line2][0][1]))<100): ###set at beginning
        neighbors[key].append(value)

In [11]:
#for key in neighbors:
    #print(key, neighbors[key])

# GUI

In [12]:
#import Tkinter as tk
#import ttk
import tkinter as tk
import tkinter.ttk

win = tk.Tk()
win.title("Set Pairs")   

#ttk.Label(win, text="Select category:").grid(column=0, row=0)   
tkinter.ttk.Label(win, text="Select category:").grid(column=0, row=0) 
#ttk.Label(win, text="content starts from").grid(column=2, row=0) 
#ttk.Label(win, text="content ends before").grid(column=3, row=0)     
pos_or_area = StringVar()
def select():  
    if pos_or_area.get() == 'area':
        ls = list(neighbors.keys())   
        ls += ['#END']
        dropdown2['values'] = ls
    else:
        ls = neighbors[category.get()]
        dropdown2['values'] = ls
    dropdown2.current()    

R1 = tkinter.ttk.Radiobutton(win, text="find by position", variable=pos_or_area, value='pos', command=select)
R1.grid(column = 2, row = 0)
R2 = tkinter.ttk.Radiobutton(win, text="find by area", variable=pos_or_area, value='area', command=select)
R2.grid(column = 3, row = 0)
    
pairs = {}
def bind():   
    #key = contents[category.get()]
    #new_value = contents[content.get()]
    key = category.get()
    if not key in pairs:
        pairs[key] = set([])
    
    if pos_or_area.get()=='pos':
        a = first_page_contents[category.get()][0]
        b = first_page_contents[content.get()][0]
        c = (b[0]-a[0], b[1]-a[1], b[2]-a[2], b[3]-a[3], b[4], pos_or_area.get())
        pairs[key].add(c)
    else:
        c = (content.get(), pos_or_area.get()) 
        pairs[key].add(c)
        
    dropdown2.current()
    
# creat a drop-down list
category = tk.StringVar()
dropdown1 = tkinter.ttk.Combobox(win, width=40, height=20, textvariable=category)
dropdown1['values'] = list(neighbors.keys())   
dropdown1.grid(column=0, row=1)      
#dropdown1.current(0)     


# create another drop-down list
content = tk.StringVar()
dropdown2 = tkinter.ttk.Combobox(win, width=40, textvariable=content)    
dropdown2.grid(column=2, row=1)  
 

# bind button
action = tkinter.ttk.Button(win, text="Bind", command=bind)
action.grid(column=4, row=1) 

win.mainloop()      # window

In [13]:
#for i in pairs:
    #print(i)
    #print(type(pairs[i]))
    #print((pairs[i]))
    #for i in pairs[i]:
        #print(len(i))

In [14]:
import re

page = 0
f = open(path2+'page'+str(page), 'r')
text = f.read()
text, num = re.subn(r'TEXT_BOX.*?height:[\d]+px;', '',text)
f.close()

reports = {}
pages = text.split('#PAGE')
for page in pages:
    if(len(page) == 0):
        continue
    page_num = page[page.find('<<<')+3:page.find('>>>')]
    page_, num = re.subn(r'<<<.*?>>>', '',page)
    reports[int(page_num)] = page_
    

In [15]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO

def pdf2txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

In [16]:
import re

text = pdf2txt(path1)

reports = {}
pages = text.split('#PAGE')
for page in pages:
    if(len(page) == 0):
        continue
    page_num = page[page.find('<<<')+3:page.find('>>>')]
    page_, num = re.subn(r'<<<.*?>>>', '',page)
    try:
        reports[int(page_num)] = page_
    except:
        print(page)

In [17]:
def find_content_by_area(text, start, end, head='CONTRACT NUMBER'): #####
    #p = re.compile(r'{%s}.*{%s}' % (start, end))
    start = start.lstrip()
    end = end.lstrip()
    content= []
    flag = False
    #for page_num, page in sorted(reports.iteritems()):
    for page_num, page in sorted(reports.items()):
        search = []
        if flag:
            flag = False
            search = re.findall('%s.*?%s' % (head, end), page, re.DOTALL)
        elif end != 'END' and start in page and end in page:
            search = re.findall('%s.*?%s' % (start, end), page, re.DOTALL)
        else:
            search = re.findall('%s.*' % start, page, re.DOTALL)
            flag = True
        for i in search:
            i = i.replace(start, '')
            i = i.replace(end, '')
            i = i.replace('\n\n', ' ')
            #content.append('#PAGE_'+str(page_num)+' '+i)
            content.append(' ' + i)
    #print(content)
    return content

In [18]:
data = {}
for cat in pairs:
    for c in pairs[cat]:
        if c[-1] == 'area':
            content = find_content_by_area(text, cat, c[0], )
            data[cat] = content

In [19]:
def find_content_by_pos(content_map, content_pos):
    content = ''
    for c in content_map:
        m = 1 #### maximal error allowed#########################
        for pos in content_map[c]:
            error = abs(content_pos[0]-pos[0]) + abs(content_pos[1]-pos[1])
            #print error
            if error < m:
                content += '#PAGE_'+pos[-1]+' '+c
    #print(m)
    return content
    
#data = {}
content_map = record_pos(path2, 0)
for cat in pairs:
    print(pairs[cat])
    #if cat in content_map:  #unnecessary??
    for pos in content_map[cat]:
        content = ''
        for shift in pairs[cat]:
            if shift[-1] == 'pos':
                content_pos = (pos[0]+shift[0], pos[1]+shift[1], pos[2]+shift[2], pos[3]+shift[3])
                content += find_content_by_pos(content_map, content_pos)
        if len(content) != 0:
            if cat in data: #unnecessary??
                data[cat].append(content)
            else:
                data[cat] = [content]
           

{('#END', 'area')}


# generate CSV table

In [30]:
#for i in data:
    #print (i)
    #for j in data[i]:
        #print (j)
#print(len(data))

In [31]:
import csv
f = open(path2+'result_py3.csv', 'w', newline = '')
writer = csv.writer(f)

for i, cat in enumerate(data):
    #if i >50:
        #break
    writer.writerow([cat] + data[cat])
    #print(i) 
    #print(cat)
f.close()

# new 

In [23]:
ehab_stop_words = []

for word in open(r"ehabs_stop_words.txt",'r'):
    ehab_stop_words.append(word.strip())

from sklearn.feature_extraction import text 

my_stop_words = text.ENGLISH_STOP_WORDS.union(ehab_stop_words) # union the ehab's stop words and the stop words in the pachage

In [24]:
print(data.keys())

dict_keys([' QC NARRATIVES'])


In [50]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from gensim.models.word2vec import Word2Vec
from nltk.stem import WordNetLemmatizer
from gensim.models.keyedvectors import KeyedVectors
from sklearn.feature_extraction.text import CountVectorizer

from nltk.stem.porter import PorterStemmer


from nltk import word_tokenize,sent_tokenize

#raw_data = pd.read_csv("out.csv", header=0, sep=None, usecols=['description'], engine='python')
def review_to_wordlist(raw_sentence, join=0):
    review_text = re.sub("[^a-zA-Z]", " ", raw_sentence)
    words = review_text.lower().split()
    stops = set(stopwords.words("english")).union(my_stop_words)
    meaningful_words = [w for w in words if not w in stops]
    porter_stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    #words = [lemmatizer.lemmatize(w) for w in meaningful_words]
    words = [porter_stemmer.stem(lemmatizer.lemmatize(w)) for w in meaningful_words]
    if join:
        return (" ".join(words))
    else:
        return words

def review_to_sentences(review):
    raw_sentences = sent_tokenize(review.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(review_to_wordlist(raw_sentence))
    return sentences

sentences = []
for review in data[' QC NARRATIVES']:
    sentences += review_to_sentences(review)

#print(sentences)

#print (len(sentences))
#print (sentences[0])
model = Word2Vec(sentences,min_count=2,sg=1, size = 300)
print(len(model.wv.vocab.keys()))
print(model.wv.vocab.keys())
print()
#model.build_vocab(features)
#print(len(model['medical']))

#word_vectors = model.wv
#word_vectors.save('similar_stopwords')

#word_vectors = KeyedVectors.load('similar')

features = ['crew','weather']
#features = ['crew','tool', 'material','shortage','weather','foreman']

selected_feature = {}

for i in features:
    similar_words = model.wv.most_similar(i)
    selected_feature[i] = similar_words
    #print(similar_words)

print(selected_feature)
#clean_data = []
#num_data = raw_data["description"].size
#for i in range(0, num_data):
    #clean_data.append(review_to_wordlist(raw_data["description"][i], join=1))


1142
dict_keys(['activ', 'progress', 'aai', 'work', 'shut', 'water', 'remov', 'refriger', 'gener', 'comment', 'control', 'inspect', 'result', 'today', 'qualiti', 'test', 'remark', 'safeti', 'meet', 'preparatori', 'initi', 'held', 'demo', 'prep', 'attach', 'minut', 'start', 'number', 'jim', 'lafluer', 'date', 'advanc', 'notic', 'finish', 'qc', 'requir', 'complet', 'qa', 'punch', 'list', 'item', 'issu', 'report', 'correct', 'contractor', 'site', 'day', 'labor', 'hour', 'classif', 'oper', 'engin', 'employ', 'asbesto', 'abat', 'total', 'equip', 'employe', 'page', 'nov', 'tuesday', 'contract', 'daca', 'weather', 'caus', 'delay', 'temperatur', 'min', 'f', 'max', 'precipit', 'mph', 'wind', 'evergreen', 'manag', 'review', 'ed', 'tree', 'preserv', 'plan', 'w', 'ith', 'coe', 'base', 'miss', 'addit', 'ere', 'ad', 'condit', 'trunk', 'cost', 'area', 'remain', 'schedul', 'wednesday', 'dow', 'n', 'ater', 'onsit', 'rfi', 'rfp', 'statu', 'updat', 'na', 'ork', 'narr', 'saturday', 'accid', 'certif', 'beh

In [27]:
for i in model.wv.vocab:
    print(i)
    print(model.wv.vocab[i])

labor
Vocab(count:1107, index:6, sample_int:989153710)
hour
Vocab(count:1146, index:5, sample_int:969477073)
reported
Vocab(count:555, index:17, sample_int:1490179342)
today
Vocab(count:1224, index:2, sample_int:933233397)
classification
Vocab(count:537, index:19, sample_int:1520342986)
operating
Vocab(count:1162, index:3, sample_int:961718688)
engineer
Vocab(count:1162, index:4, sample_int:961718688)
employer
Vocab(count:537, index:20, sample_int:1520342986)
asbestos
Vocab(count:137, index:85, sample_int:3650051895)
abatement
Vocab(count:137, index:86, sample_int:3650051895)
total
Vocab(count:441, index:24, sample_int:1715359409)
worked
Vocab(count:202, index:65, sample_int:2818024602)
date
Vocab(count:425, index:29, sample_int:1754982620)
w
Vocab(count:586, index:14, sample_int:1441898559)
orked
Vocab(count:390, index:38, sample_int:1851148346)
number
Vocab(count:393, index:37, sample_int:1842338721)
employee
Vocab(count:201, index:66, sample_int:2827210486)
laborer
Vocab(count:1310,

In [29]:
model.wv['crew']

array([ -1.80744007e-02,   1.54573424e-02,  -1.09833896e-01,
        -8.73168558e-02,  -7.95951933e-02,   6.57372400e-02,
        -3.75796184e-02,   3.99849638e-02,  -1.75310075e-01,
        -4.25117724e-02,  -7.80971497e-02,  -9.68174934e-02,
        -1.47007257e-02,   5.52864447e-02,   1.52264982e-01,
         1.69443980e-01,   6.96560964e-02,  -1.44950077e-02,
        -1.85099058e-02,  -2.03431491e-02,   1.27419438e-02,
         2.27716435e-02,   1.30984321e-01,   4.08432931e-02,
         7.34004453e-02,  -2.32358724e-01,   1.22837156e-01,
         1.59608722e-01,  -3.50018479e-02,  -3.27849351e-02,
         1.42360479e-01,   2.39776149e-01,  -2.63327301e-01,
         2.32543603e-01,  -2.76361965e-02,   3.55679616e-02,
         2.14405925e-04,  -2.44925573e-01,   1.52654722e-01,
         7.28860348e-02,  -5.69542572e-02,  -1.93098307e-01,
         2.07424033e-02,   9.11744833e-02,  -1.00702658e-01,
        -2.33313590e-02,   1.47083551e-01,  -9.16301981e-02,
        -9.60987955e-02,

In [51]:

features = [["Footings", "Continuous"], ["Footings", "Wall"],["Footings", "Foundations"],["Keyway"], ["Footings","Formwork"], ["Footings","Steel"], ["Footings"], ["Concrete"], ["Footings"], ["Screed"],
["Foundation", "Walls"], ["Foundation", "Pilasters"], ["Walls","Pilasters"],["Concrete", "Masonry", "Foundation"], ["CMU", "Foundation"],
["Excavating", "Wall"], ["Excavating", "Foundations"],["Backfilling", "Wall"], ["Backfilling","Foundations"],["Perimeter", "Insulation"], ["Hauling", "Foundations"]]
#features = ['crew','tool', 'material','shortage','weather','foreman']

porter_stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
#words = [lemmatizer.lemmatize(w) for w in meaningful_words]

for i in range(0, len(features)):
    features[i] = [porter_stemmer.stem(lemmatizer.lemmatize(w)) for w in features[i]]


selected_feature = {}

for n, i in enumerate(features):
    try:
        similar_words = model.wv.most_similar(i)
        selected_feature[n] = similar_words
    except:
        pass
    
    #print(similar_words)

print(selected_feature)
#clean_data = []
#num_data = raw_data["description"].size
#for i in range(0, num_data):
    #clean_data.append(review_to_wordlist(raw_data["description"][i], join=1))

{0: [('clearanc', 0.9326205253601074), ('lay', 0.9270715117454529), ('stake', 0.9248197674751282), ('pavillion', 0.9226517677307129), ('strap', 0.9207562804222107), ('feeder', 0.9120868444442749), ('stripe', 0.911787748336792), ('ramp', 0.9103425741195679), ('roy', 0.9077193140983582), ('hickey', 0.9070169925689697)], 1: [('baxter', 0.8769237995147705), ('green', 0.8730360269546509), ('art', 0.8603296875953674), ('vanit', 0.8525203466415405), ('set', 0.851000964641571), ('plate', 0.8467895984649658), ('bungalow', 0.8461379408836365), ('procur', 0.8450535535812378), ('elect', 0.8438275456428528), ('block', 0.8397574424743652)], 2: [('bar', 0.9610644578933716), ('formw', 0.9602829217910767), ('stem', 0.9422202110290527), ('bungalow', 0.9383507966995239), ('imb', 0.9303990006446838), ('tie', 0.9270659685134888), ('baxter', 0.9231873750686646), ('grind', 0.9193376302719116), ('pavillion', 0.917925238609314), ('rebar', 0.9098331928253174)], 5: [('stripe', 0.9297739267349243), ('lay', 0.9265

In [52]:
for i in selected_feature:
    print(features[i])
    for j in selected_feature[i]:
        print(j[0])

['foot', 'continu']
clearanc
lay
stake
pavillion
strap
feeder
stripe
ramp
roy
hickey
['foot', 'wall']
baxter
green
art
vanit
set
plate
bungalow
procur
elect
block
['foot', 'foundat']
bar
formw
stem
bungalow
imb
tie
baxter
grind
pavillion
rebar
['foot', 'steel']
stripe
lay
bolt
emb
reinforc
bois
axelson
fetch
late
feeder
['foot']
tie
pour
bungalow
form
strip
bar
rebar
formw
axelsen
baxter
['concret']
slump
placement
mti
air
soil
cylind
densiti
cid
prior
tese
['foot']
tie
pour
bungalow
form
strip
bar
rebar
formw
axelsen
baxter
['foundat', 'wall']
art
craft
deck
clip
current
vanit
add
gwb
rest
ch
['concret', 'masonri', 'foundat']
trash
pavillion
vent
paper
enclosur
mesh
tar
flatw
veneer
cancel
['excav', 'wall']
vanit
run
art
continu
elect
vaniti
windo
betw
backfil
joint
['excav', 'foundat']
tw
proctor
eek
stake
hich
pavillion
asphalt
cancel
reject
larg
['backfil', 'wall']
vanit
windo
art
vaniti
flash
applianc
overhead
counter
cultur
elect
['backfil', 'foundat']
tw
rock
sleev
gutter
lb
pap

In [49]:

features = [["Footings", "Continuous"], ["Footings", "Wall"],["Footings", "Foundations"],["Keyway"], ["Footings","Formwork"], ["Footings","Steel"], ["Footings"], ["Concrete"], ["Footings"], ["Screed"],
["Foundation", "Walls"], ["Foundation", "Pilasters"], ["Walls","Pilasters"],["Concrete", "Masonry", "Foundation"], ["CMU", "Foundation"],
["Excavating", "Wall"], ["Excavating", "Foundations"],["Backfilling", "Wall"], ["Backfilling","Foundations"],["Perimeter", "Insulation"], ["Hauling", "Foundations"]]
#features = ['crew','tool', 'material','shortage','weather','foreman']

porter_stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
#words = [lemmatizer.lemmatize(w) for w in meaningful_words]

for i in range(0, len(features)):
    features[i] = [lemmatizer.lemmatize(w).lower() for w in features[i]]
    print(features[i])


['footings', 'continuous']
['footings', 'wall']
['footings', 'foundations']
['keyway']
['footings', 'formwork']
['footings', 'steel']
['footings']
['concrete']
['footings']
['screed']
['foundation', 'walls']
['foundation', 'pilasters']
['walls', 'pilasters']
['concrete', 'masonry', 'foundation']
['cmu', 'foundation']
['excavating', 'wall']
['excavating', 'foundations']
['backfilling', 'wall']
['backfilling', 'foundations']
['perimeter', 'insulation']
['hauling', 'foundations']
